In [2]:
# filter dataset
!python3 MT-Preparation/filtering/filter.py ./en-zh.en ./en-zh.zh en zh

Dataframe shape (rows, columns): (231267, 2)
--- Rows with Empty Cells Deleted	--> Rows: 231267
--- Duplicates Deleted			--> Rows: 229646
--- Source-Copied Rows Deleted		--> Rows: 229640
--- Too Long Source/Target Deleted	--> Rows: 229604
--- HTML Removed			--> Rows: 229604
--- Rows will remain true-cased		--> Rows: 229604
--- Rows with Empty Cells Deleted	--> Rows: 229604
--- Rows Shuffled			--> Rows: 229604
--- Source Saved: ./en-zh.zh-filtered.zh
--- Target Saved: ./en-zh.en-filtered.en


In [3]:
# train a sentencepiece model for subwording
!python3 MT-Preparation/subwording/1-train_unigram.py ./en-zh.en-filtered.en ./en-zh.zh-filtered.zh

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=./en-zh.zh-filtered.zh --model_prefix=source --vocab_size=50000 --hard_vocab_limit=false --split_digits=true
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./en-zh.zh-filtered.zh
  input_format: 
  model_prefix: source
  model_type: UNIGRAM
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 0
  use_all_vocab: 

In [5]:
# subword the dataset
!python3 MT-Preparation/subwording/2-subword.py source.model target.model ./en-zh.en-filtered.en ./en-zh.zh-filtered.zh

Source Model: source.model
Target Model: target.model
Source Dataset: ./en-zh.zh-filtered.zh
Target Dataset: ./en-zh.en-filtered.en
Done subwording the source file! Output: ./en-zh.zh-filtered.zh.subword
Done subwording the target file! Output: ./en-zh.en-filtered.en.subword


In [6]:
# first 3 lines before subwording
!head -n 3 ./en-zh.en-filtered.en && echo "-----" && head -n 3 ./en-zh.zh-filtered.zh

我们不使用化石能源
正如你所知的， 我们让免疫系统对这个奇怪的分子产生了反应， 而这个分子并不是我们做的，我们能在其他动物和物体上看到这样的物质。
这么多年来，这个国家 因政治和宗教原因支离破碎
-----
We are not using fossil energies.
Now, I was thinking about that and I said, you know, we've got this immune response to this ridiculous molecule that we don't make, and we see it a lot in other animals and stuff.
For years, the country has been divided between politics and religion.


In [7]:
# first 3 lines after subwording
!head -n 3 ./en-zh.en-filtered.en.subword && echo "---" && head -n 3 ./en-zh.zh-filtered.zh.subword after

▁我们 不使用 化石 能源
▁正如你 所知的 , ▁我们让 免疫系统 对这个 奇怪的 分子 产生了 反应 , ▁而这个 分子 并不是 我们做的 , 我们能 在 其他动物 和 物体上 看到这样的 物质 。
▁ 这么多年来 , 这个国家 ▁因 政治和 宗教 原因 支离破碎
---
==> ./en-zh.en-filtered.en.subword <==
▁We ▁are ▁not ▁using ▁fossil ▁energies .
▁Now , ▁I ▁was ▁thinking ▁about ▁that ▁and ▁I ▁said , ▁you ▁know , ▁we ' ve ▁got ▁this ▁immune ▁response ▁to ▁this ▁ridiculous ▁molecule ▁that ▁we ▁don ' t ▁make , ▁and ▁we ▁see ▁it ▁a ▁lot ▁in ▁other ▁animals ▁and ▁stuff .
▁For ▁years , ▁the ▁country ▁has ▁been ▁divided ▁between ▁politics ▁and ▁religion .
head: after: No such file or directory


In [10]:
# split the dataset into training set, development set, and test set
# Development and test sets should be between 1000 and 5000 segments (here we chose 200)
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py 2000 2000 ./en-zh.en-filtered.en.subword ./en-zh.zh-filtered.zh.subword

Dataframe shape: (229604, 2)
--- Empty Cells Deleted --> Rows: 229604
--- Wrote Files
Done!
Output files
./en-zh.zh-filtered.zh.subword.train
./en-zh.en-filtered.en.subword.train
./en-zh.zh-filtered.zh.subword.dev
./en-zh.en-filtered.en.subword.dev
./en-zh.zh-filtered.zh.subword.test
./en-zh.en-filtered.en.subword.test


In [11]:
!wc -l ./*.subword.*

    2000 ./en-zh.en-filtered.en.subword.dev
    2000 ./en-zh.en-filtered.en.subword.test
     200 ./en-zh.en-filtered.en.subword.test.desubword
  225604 ./en-zh.en-filtered.en.subword.train
     200 ./en-zh.en-filtered.en.subword.translated
     200 ./en-zh.en-filtered.en.subword.translated.desubword
    2000 ./en-zh.zh-filtered.zh.subword.dev
    2000 ./en-zh.zh-filtered.zh.subword.test
  225604 ./en-zh.zh-filtered.zh.subword.train
  459808 total


In [12]:
# check the first and last line from each dataset
!echo "---First line---"
!head -n 1 ./*.{train,dev,test}

!echo -e "\n---Last line---"
!tail -n 1 ./*.{train,dev,test}

---First line---
==> ./en-zh.en-filtered.en.subword.train <==
▁We ▁are ▁not ▁using ▁fossil ▁energies .

==> ./en-zh.zh-filtered.zh.subword.train <==
▁我们 不使用 化石 能源

==> ./en-zh.en-filtered.en.subword.dev <==
▁I ▁can ' t ▁take ▁credit ▁for ▁Christopher ' s ▁journey ▁to ▁success , ▁but ▁I ▁certainly ▁did ▁my ▁part ▁to ▁keep ▁him ▁on ▁the ▁path .

==> ./en-zh.zh-filtered.zh.subword.dev <==
▁我不能 说 克里斯托弗 的成功 完全是 因为我 , ▁但我 的确 将他 带到了 正确 的道路上 。

==> ./en-zh.en-filtered.en.subword.test <==
▁Everybody ▁wins .

==> ./en-zh.zh-filtered.zh.subword.test <==
▁ 共 赢

---Last line---
==> ./en-zh.en-filtered.en.subword.train <==
▁This ▁was ▁a ▁headline ▁in ▁a ▁U . K . ▁newspaper , ▁The ▁Guardian , ▁not ▁that ▁long ▁ago .

==> ./en-zh.zh-filtered.zh.subword.train <==
▁这句话 是一家 英国 报纸 的头条 , ▁不久 之前 出版的 《 卫报 》。

==> ./en-zh.en-filtered.en.subword.dev <==
▁But ▁anyway , ▁the ▁council ▁granted ▁these ▁people ▁permission , ▁and ▁they ' re ▁now ▁trying ▁to ▁build ▁their ▁cemetery .

==> ./en-zh.zh-filtered.zh.subwo